In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='ticks', font_scale=1.5)

import warnings
warnings.filterwarnings('ignore')

In [37]:
df = pd.read_pickle('c:/Users/Shushant PC/Desktop/Data Analysis Practice/data/earningsCall/motley-fool-data.pkl')

In [38]:
df.head()

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [39]:
# remove the text that appears after : in exchange column
df['exchange'] = df['exchange'].str.split(':').str[0]

# remove the information that appear before the date in all rows
df['date'] = df.iloc[:, 0].apply(lambda x: x[-1] if isinstance(x, list) else x)
# remove similar  str "...... Earnings  Call" from the first column using regex
target_words = ['Call', 'Transcript']
df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'^.*?(?:' + '|'.join(target_words) + r')\s*', '', regex=True)
# convert the date string to datetime object
df['date'] = pd.to_datetime(df['date'], format='mixed', utc=True)
df.head()

,date,exchange,q,ticker,transcript
0,2020-08-27 21:00:00+00:00,NASDAQ,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,2020-07-30 16:30:00+00:00,NYSE,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,2019-10-23 17:00:00+00:00,NASDAQ,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,2019-11-06 12:00:00+00:00,NASDAQ,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,2019-08-07 08:30:00+00:00,NASDAQ,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [40]:
# replace text in the exchange column with the first letter of the text
df['exchange'] = df['exchange'].str.replace('NYSEMKT', 'NYSE', regex=False)
df['exchange'] = df['exchange'].str.replace('(NASDAQ', 'NASDAQ', regex=False)
df['exchange'] = df['exchange'].str.replace('(NYSE', 'NYSE', regex=False)

df['exchange'].value_counts().sort_index(ascending=False)

exchange
TSX           2
OTC          34
NYSE      10294
NASDAQ     8425
Name: count, dtype: int64

In [41]:
df['date'].fillna(method='ffill', inplace=True)

In [42]:
print("DATASET OVERVIEW:")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Unique companies: {df['ticker'].nunique()}")
print(f"Unique exchanges: {df['exchange'].unique()}")
print(f"Total earnings calls: {len(df)}")

DATASET OVERVIEW:
Date range: 2017-11-03 08:30:00+00:00 to 2023-02-23 11:00:00+00:00
Unique companies: 2876
Unique exchanges: ['NASDAQ' 'NYSE' 'OTC' 'TSX']
Total earnings calls: 18755


In [43]:
df['year'] = pd.to_datetime(df['date']).dt.year
df['quarter'] = df['q'].str.split('-').str[-1]

In [44]:
print("\nDISTRIBUTION BY YEAR:")
year_counts = df['year'].value_counts().sort_index()
print(year_counts)

print("\nDISTRIBUTION BY QUARTER:")
quarter_counts = df['quarter'].value_counts().sort_index()
print(quarter_counts)


DISTRIBUTION BY YEAR:
year
2017       9
2018     141
2019    1523
2020    2900
2021    9685
2022    4295
2023     202
Name: count, dtype: int64

DISTRIBUTION BY QUARTER:
quarter
Q1    4730
Q2    5071
Q3    4481
Q4    4473
Name: count, dtype: int64


In [45]:
print("\nTOP 20 COMPANIES BY EARNINGS CALLS:")
top_companies = df['ticker'].value_counts().head(20)
print(top_companies)


TOP 20 COMPANIES BY EARNINGS CALLS:
ticker
AAPL     62
GOOGL    58
TSLA     52
AMZN     39
DIS      38
BEST     37
SE       27
MRNA     21
PTC      21
PARA     18
WWE      18
IBM      18
SQ       17
BLK      17
KO       17
INTC     16
YY       16
CLDT     16
WMT      15
AXL      15
Name: count, dtype: int64


In [46]:
# Sample transcript length analysis
df['transcript_length'] = df['transcript'].str.len()
print(f"\nTRANSCRIPT LENGTH STATS:")
print(f"Mean length: {df['transcript_length'].mean():.0f} characters")
print(f"Median length: {df['transcript_length'].median():.0f} characters")
print(f"Min length: {df['transcript_length'].min():.0f} characters")
print(f"Max length: {df['transcript_length'].max():.0f} characters")


TRANSCRIPT LENGTH STATS:
Mean length: 47303 characters
Median length: 47930 characters
Min length: 3616 characters
Max length: 188514 characters


In [47]:
# Show sample transcript structure
print("\nSAMPLE TRANSCRIPT PREVIEW:")
sample_transcript = df.iloc[0]['transcript'][:1000]
print(f"Company: {df.iloc[0]['ticker']}")
print(f"Date: {df.iloc[0]['date']}")
print(f"First 1000 characters:\n{sample_transcript}...")


SAMPLE TRANSCRIPT PREVIEW:
Company: BILI
Date: 2020-08-27 21:00:00+00:00
First 1000 characters:
Prepared Remarks:
Operator
Good day, and welcome to the Bilibili 2020 Second Quarter Earnings Conference Call. Today's conference is being recorded.
At this time, I would like to turn the conference over to Juliet Yang, Senior Director of Investor Relations. Please go ahead.
Juliet Yang -- Senior Director of Investor Relations
Thank you, operator.
Please note the discussion today will contain forward-looking statements relating to the Company's future performance, and are intended to qualify for the Safe Harbor from liability, as established by the US Private Securities Litigation Reform Act. Such statements are not guarantees of future performance and are subject to certain risks and uncertainties, assumptions and other factors. Some of these risks are beyond the Company's control and could cause actual results to differ materially from those mentioned in today's press release and this dis

In [48]:
# Check for very short transcripts (might be incomplete)
short_transcripts = df[df['transcript_length'] < 5000]
print(f"Transcripts < 5000 chars: {len(short_transcripts)} ({len(short_transcripts)/len(df)*100:.1f}%)")

Transcripts < 5000 chars: 5 (0.0%)


In [49]:
duplicates = df.duplicated(subset=['ticker', 'date']).sum()
print(f"Duplicate ticker-date combinations: {duplicates}")

Duplicate ticker-date combinations: 1211


In [15]:
df.drop_duplicates(subset=['ticker','date'], keep='last', inplace=True, ignore_index=True)

In [16]:
df.shape

(17544, 8)

In [17]:
df[df['ticker']=='NVDA']

,date,exchange,q,ticker,transcript,year,quarter,transcript_length
1038,2018-11-15 17:30:00+00:00,NASDAQ,2019-Q3,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2018,Q3,41989
2614,2021-02-24 17:00:00+00:00,NASDAQ,2021-Q4,NVDA,"Prepared Remarks:\nOperator\nGood afternoon, m...",2021,Q4,51612
5748,2019-11-14 17:30:00+00:00,NASDAQ,2019-Q3,NVDA,"Prepared Remarks:\nOperator\nGood afternoon, m...",2019,Q3,49730
6048,2019-08-15 17:30:00+00:00,NASDAQ,2020-Q2,NVDA,"Prepared Remarks:\nOperator\nGood afternoon, m...",2019,Q2,50134
11442,2021-11-17 17:00:00+00:00,NASDAQ,2021-Q3,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2021,Q3,50679
12240,2022-11-16 17:00:00+00:00,NASDAQ,2023-Q3,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2022,Q3,49369
15460,2022-05-25 17:00:00+00:00,NASDAQ,2023-Q1,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2022,Q1,51261
15822,2022-02-16 17:30:00+00:00,NASDAQ,2022-Q4,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2022,Q4,48938
16748,2021-08-18 17:00:00+00:00,NASDAQ,2022-Q2,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2021,Q2,54240
17536,2022-08-24 17:00:00+00:00,NASDAQ,2023-Q2,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,2022,Q2,50275


In [18]:
target_companies = {
'AAPL': 'Apple',
'GOOGL': 'Google',
'TSLA': 'Tesla',
'AMZN': 'Amazon',
'DIS': 'Disney',
'SE': 'Sea Limited',
'NVDA': 'Nvidia',
'PTC': 'PTC',
'PARA': 'Paramount',
'META': 'Meta',
'IBM': 'IBM'}

In [19]:
data = df[df['ticker'].isin(target_companies)]
data = data[['date','exchange','ticker','transcript','year','quarter','transcript_length']]

In [20]:
data.shape

(123, 8)

In [23]:
data.head()

,date,exchange,ticker,transcript,year,quarter,transcript_length
103,2022-10-27 17:30:00+00:00,NASDAQ,AMZN,Prepared Remarks:\nOperator\nThank you for sta...,2022,Q3,30798
143,2021-05-13 16:30:00+00:00,NYSE,DIS,"Prepared Remarks:\nOperator\nGood day, and tha...",2021,Q2,54250
213,2020-01-22 17:00:00+00:00,NASDAQ,PTC,"Prepared Remarks:\nOperator\nGood afternoon, l...",2020,Q1,52251
214,2022-07-18 17:00:00+00:00,NYSE,IBM,"Prepared Remarks:\nOperator\nWelcome, and than...",2022,Q2,55209
389,2022-07-28 17:00:00+00:00,NASDAQ,AAPL,"Prepared Remarks:\nOperator\nGood day, and wel...",2022,Q3,48337


In [24]:
data['ticker'].value_counts()

ticker
GOOGL    15
AAPL     14
AMZN     12
DIS      12
TSLA     12
PTC      11
IBM      11
NVDA     10
SE       10
META      8
PARA      8
Name: count, dtype: int64

In [25]:
data['company_name'] = data['ticker'].map(target_companies)

In [26]:
data.head()

,date,exchange,ticker,transcript,year,quarter,transcript_length,company_name
103,2022-10-27 17:30:00+00:00,NASDAQ,AMZN,Prepared Remarks:\nOperator\nThank you for sta...,2022,Q3,30798,Amazon
143,2021-05-13 16:30:00+00:00,NYSE,DIS,"Prepared Remarks:\nOperator\nGood day, and tha...",2021,Q2,54250,Disney
213,2020-01-22 17:00:00+00:00,NASDAQ,PTC,"Prepared Remarks:\nOperator\nGood afternoon, l...",2020,Q1,52251,PTC
214,2022-07-18 17:00:00+00:00,NYSE,IBM,"Prepared Remarks:\nOperator\nWelcome, and than...",2022,Q2,55209,IBM
389,2022-07-28 17:00:00+00:00,NASDAQ,AAPL,"Prepared Remarks:\nOperator\nGood day, and wel...",2022,Q3,48337,Apple


In [27]:
data['company_name'].value_counts()

company_name
Google         15
Apple          14
Amazon         12
Disney         12
Tesla          12
PTC            11
IBM            11
Nvidia         10
Sea Limited    10
Meta            8
Paramount       8
Name: count, dtype: int64

In [28]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to C:\Users\Shushant
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Shushant PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [60]:
# from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import openai 
from dotenv import load_dotenv
import os
import shutil
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [30]:
load_dotenv(verbose=True)

True

In [31]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [32]:
embedding_fn = OpenAIEmbeddings(model="text-embedding-3-large")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150
)

In [33]:
documents = []

for idx, row in data.head(10).iterrows():
    chunks = text_splitter.split_text(row["transcript"])
    for chunk in chunks:
        doc = Document(
            page_content=chunk,
            metadata={
                "company": row["company_name"],
                "quarter": row["quarter"],
                "year": int(row["year"]),
            }
        )
        documents.append(doc)

In [51]:
documents[:5]

[Document(metadata={'company': 'Amazon', 'quarter': 'Q3', 'year': 2022}, page_content="Prepared Remarks:\nOperator\nThank you for standing by. Good day, everyone, and welcome to the Amazon.com Q3 2022 financial results teleconference. [Operator instructions] Today's call is being recorded. For opening remarks, I will be turning the call over to the vice president of investor relations, Dave Fildes.\nThank you, sir. Please go ahead.\nDave Fildes -- Vice President, Investor Relations"),
 Document(metadata={'company': 'Amazon', 'quarter': 'Q3', 'year': 2022}, page_content="Hello, and welcome to our Q3 2022 financial results conference call. Joining us today to answer your questions is Brian Olsavsky, our CFO. As you listen to today's conference call, we encourage you to have our press release in front of you, which includes our financial results, as well as metrics and commentary on the quarter. Please note, unless otherwise stated, all comparisons in this call will be against our results

In [52]:
vectorstore = Chroma(
    collection_name="earnings_calls",
    embedding_function=embedding_fn,
    persist_directory="./chroma_store"
)

vectorstore.add_documents(documents)
vectorstore.persist()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
C:\Users\Shushant PC\AppData\Local\Temp\ipykernel_10884\1847745746.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [55]:
retriever = vectorstore.as_retriever(search_kwargs={
    "k": 5,
    "filter": {"$and": [{"company": "Tesla"}, {"quarter": "Q2"}, {"year": 2021}]}
})

docs = retriever.get_relevant_documents("What was Tesla's gross margin in Q2 2021?")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [56]:
docs

[Document(metadata={'company': 'Tesla', 'quarter': 'Q2', 'year': 2021}, page_content="Elon Musk -- Chief Executive Officer\nYeah. I mean, any given price is going to be wrong, so we'll just adjust it over time as we see the value proposition makes sense to people. So we're just really -- I'm not thinking about this a lot right now. We need to make Full Self-Driving work in order for it to be a compelling value proposition."),
 Document(metadata={'company': 'Tesla', 'quarter': 'Q2', 'year': 2021}, page_content="Andrew Baglino -- Senior Vice President, Powertrain and Energy Engineering\nYeah, 10%. Closer to 10%\nElon Musk -- Chief Executive Officer\nYes, so it's like people don't think like this is like the Messiah or something, wet versus dry reduces. To dry is like 10% less cost than wet. So it's not 10% slow, nothing to sneeze at, especially if you're making hundreds of gigawatt hours a year. But it's not the Messiah, basically.\nAndrew Baglino -- Senior Vice President, Powertrain and

In [71]:
results = vectorstore.similarity_search_with_relevance_scores("What was Tesla's gross margin in Q2 2021?", k=5,filter = {"$and": [{"company": "Tesla"}, {"quarter": "Q2"}, {"year": 2021}]})

In [72]:
results

[(Document(metadata={'company': 'Tesla', 'quarter': 'Q2', 'year': 2021}, page_content='This volume increase was made possible by a steady decrease in ASPs of more than 10%, driven by a road map to increase affordability and shifting mix toward our more affordable vehicles. Yet over that same period of time, our auto gross margin, excluding credit, has increased nearly 10 percentage points to our highest yet since the introduction of Model 3. This is only possible because our average cost per vehicle has reduced by more than the reduction in average price. This is a remarkable'),
  0.38807471121105985),
 (Document(metadata={'company': 'Tesla', 'quarter': 'Q2', 'year': 2021}, page_content='This was primarily driven by better cost optimization across our factories, good execution against our cost reduction plan, as well as increases in production and delivery volumes. There was some benefit from pricing actions, mostly in North America. However, it was small in the context of the other co

In [73]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based only on the above context but in proper detail: {question}
"""

In [75]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question="What was Tesla's gross margin in Q2 2021?")
print(prompt)

Human: 
Answer the question based only on the following context:

This volume increase was made possible by a steady decrease in ASPs of more than 10%, driven by a road map to increase affordability and shifting mix toward our more affordable vehicles. Yet over that same period of time, our auto gross margin, excluding credit, has increased nearly 10 percentage points to our highest yet since the introduction of Model 3. This is only possible because our average cost per vehicle has reduced by more than the reduction in average price. This is a remarkable

---

This was primarily driven by better cost optimization across our factories, good execution against our cost reduction plan, as well as increases in production and delivery volumes. There was some benefit from pricing actions, mostly in North America. However, it was small in the context of the other contributors. Note that the Model S and X program was at a slight loss for the quarter due to the relatively low volume.

---

Than

In [76]:
model = ChatOpenAI()
response_text = model.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: In Q2 2021, Tesla's auto gross profit and margin, excluding credits, increased substantially. The auto gross margin, excluding credit, has increased nearly 10 percentage points to their highest yet since the introduction of Model 3. This indicates that Tesla's gross margin in Q2 2021 was significantly higher compared to previous quarters. While the exact percentage is not provided in the context, it is clear that Tesla experienced a strong improvement in their gross margin during that quarter.
Sources: [None, None, None, None, None]
